In [1]:
from xgboost import XGBClassifier
from xgboost import DMatrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn import decomposition
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.svm import SVC
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
%pylab inline
from pandas.tools.plotting import scatter_matrix

Populating the interactive namespace from numpy and matplotlib


In [2]:
result = pd.read_csv("/Users/alexegg/Downloads/panel_stores_products_demos_8_11.csv")

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df=result
df.shape
df.head()

,panid,week,minute,units,outlet,dollars,iri_key,colupc,ou,est_acv,...,race2,race3,microwave,device_type,zipcode,fipscode,market_based_upon_zipcode,iri_geography_number,ext_fact,cnt
0,1116608,1607,9587,2,DK,4.38,642166,11600014690,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
1,1408492,1632,8079,1,DK,1.12,642166,11600015840,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
2,1809632,1584,6998,2,DK,2.72,642166,11600015840,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
3,1809632,1604,6626,4,DK,5.32,642166,11600015990,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
4,1163584,1615,651,2,DK,3.00,642166,11600016490,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4


In [4]:
# df.set_index("panid", inplace=True)
df.fillna(method="ffill", inplace=True)

In [5]:
df.head()

,panid,week,minute,units,outlet,dollars,iri_key,colupc,ou,est_acv,...,race2,race3,microwave,device_type,zipcode,fipscode,market_based_upon_zipcode,iri_geography_number,ext_fact,cnt
0,1116608,1607,9587,2,DK,4.38,642166,11600014690,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
1,1408492,1632,8079,1,DK,1.12,642166,11600015840,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
2,1809632,1584,6998,2,DK,2.72,642166,11600015840,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
3,1809632,1604,6626,4,DK,5.32,642166,11600015990,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
4,1163584,1615,651,2,DK,3.00,642166,11600016490,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4


In [6]:
print df.outlet.value_counts()
outlets=pd.get_dummies(df.l2)
print list(df.columns)

GK    452594
DK      5871
MK      2560
Name: outlet, dtype: int64
['panid', 'week', 'minute', 'units', 'outlet', 'dollars', 'iri_key', 'colupc', 'ou', 'est_acv', 'marketname', 'open', 'clsd', 'upc2', 'l1', 'l2', 'l3', 'l4', 'l5', 'vol_eq', 'product_type', 'events', 'year', 'week_number', 'quarter', 'day_of_week', 'minute_by_day', 'hour', 'time_of_day', 'time_of_day_2', 'weekday_weekend', 'panelist_id', 'panelist_type', 'combined_pre_tax_income_of_hh', 'family_size', 'household_head_race', 'type_of_residential_possession', 'county', 'age_group_applied_to_household_head', 'education_level_reached_by_household_head', 'occupation_code_of_household_head', 'age_group_applied_to_male_hh', 'education_level_reached_by_male_hh', 'occupation_code_of_male_hh', 'male_working_hour_code', 'male_smoke', 'age_group_applied_to_female_hh', 'education_level_reached_by_female_hh', 'occupation_code_of_female_hh', 'female_working_hour_code', 'fem_smoke', 'number_of_dogs', 'number_of_cats', 'children_group_co

If I'm a male of this demographic `D` and i'm in store `iri_key` on `day_of_week` at `time_of_day`, I will probably by `l2`

* If I remove `year` I drop 6% in accuracy. However, i think I should remove this from the training set b/c I will never test against previous years...
* if I remove `HISP_CAT` I drop 4%, but I have to b/c it's all nill
* `vol_eq` contributres about 38% accuracy

In [82]:
feature_list=[ 'iri_key', 
               'year', 'week_number', 'quarter', 'day_of_week', 
               'minute_by_day', 'hour', 'time_of_day', 'time_of_day_2', 'weekday_weekend', 
               'panelist_type', 
               'combined_pre_tax_income_of_hh', 'family_size', 'household_head_race', 
               'type_of_residential_possession', 'age_group_applied_to_household_head', 
               'education_level_reached_by_household_head',
               'occupation_code_of_household_head', 'age_group_applied_to_male_hh', 
               'education_level_reached_by_male_hh', 'occupation_code_of_male_hh', 
               'male_working_hour_code', 'male_smoke', 'age_group_applied_to_female_hh', 
               'education_level_reached_by_female_hh', 'occupation_code_of_female_hh',
               'female_working_hour_code', 'fem_smoke', 'number_of_dogs',
               'number_of_cats', 'children_group_code', 'marital_status', 
               'hh_lang', 'all_tvs', 'cabl_tvs',
               'race3', 'zipcode']
features = df[feature_list]

#target encoder
target_column_name="l2"
target_column = df[target_column_name]
le = preprocessing.LabelEncoder()
le.fit(list(set(target_column.values)))
target = le.transform(target_column.values)


In [83]:
features2=features.fillna(method="bfill")
oh=pd.get_dummies(features2)


features_3 = oh

In [84]:
scaled = scale(features_3)
# X_new = SelectKBest(chi2, k=2).fit_transform(scaled, target)
# X_new.shape


In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(scaled, target, test_size=0.25, random_state=4)
print (X_train.shape)

(345768, 53)


so now I can predict what somebody will buy if they if they walk into 

In [86]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, Y_train)
# print dtree.score(X_train, Y_train)
print dtree.score(X_test, Y_test)
#.377

0.385737959517


In [87]:
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
rf.score(X_test, np.ravel(Y_test))
#.415

0.42446011955889879

In [14]:
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=3)
# neigh.fit(X_train, Y_train)
# dtree.score(X_test, Y_test)

In [15]:
# from sklearn import svm
# clf = svm.LinearSVC()
# clf.fit(X_train, Y_train)